## QAOA Mixer Comparison: Vanilla, Free, and Orbit

This notebook compares three QAOA variants on the **house graph** (5 nodes, 6 edges):

| Variant | Problem | Mixer | γ / layer | β / layer |
|---------|---------|-------|-----------|----------|
| **Vanilla** | `MaxCut` | `X` | 1 | 1 |
| **Free** | `MaxCut` | `XMultiAngle` | 1 | N (one per qubit) |
| **Orbit** | `MaxCutOrbit` | `X` | K (one per edge orbit) | 1 |

- **Vanilla QAOA** uses the standard X mixer with a single shared γ and β per layer.
- **Free QAOA** (multi-angle) gives each qubit its own independent β, increasing expressibility.
- **Orbit QAOA** exploits the graph's automorphism group: edges in the same symmetry orbit share a γ parameter, enabling the optimizer to differentiate structurally distinct edge types without the full multi-angle overhead.

In [ ]:
from qaoa import QAOA, problems, mixers, initialstates

In [ ]:
import numpy as np
import networkx as nx
import sys
import matplotlib.pyplot as plt

sys.path.append("../")
from plotroutines import *

### House Graph

The house graph has 5 nodes and 6 edges arranged as a square with a triangular roof.

In [ ]:
G = nx.house_graph()

pos = {0: (0, 0), 1: (1, 0), 2: (0, 1), 3: (1, 1), 4: (0.5, 2)}
nx.draw_networkx(G, pos=pos)
plt.title("House graph")
plt.axis("off")
plt.show()

print(f"Nodes: {list(G.nodes())}")
print(f"Edges: {list(G.edges())}")

### Compute the Optimal Cut

Brute-force the minimum cost (maximum cut) so we can calculate approximation ratios.

In [ ]:
problem_ref = problems.MaxCut(G)
min_cost, max_cost = problem_ref.computeMinMaxCosts()
# cost() returns a positive value; the QAOA minimizes its negative
mincost = -min_cost   # most negative expectation value = best cut
maxcost = 0

print(f"Maximum cut value : {-mincost}")
print(f"mincost used for approximation ratio: {mincost}")

### Edge Orbits of the House Graph

The orbit QAOA assigns one γ parameter per edge orbit of the graph's automorphism group.

In [ ]:
orbit_problem = problems.MaxCutOrbit(G)
print(f"Number of edge orbits: {orbit_problem.get_num_parameters()}")
for i, orbit in enumerate(orbit_problem.edge_orbits):
    print(f"  Orbit {i}: {orbit}")

### Create QAOA Instances

In [ ]:
# Vanilla QAOA: X mixer, single γ and β per layer
qaoa_vanilla = QAOA(
    problem=problems.MaxCut(G),
    mixer=mixers.X(),
    initialstate=initialstates.Plus(),
)

# Free QAOA: XMultiAngle mixer, one β per qubit
qaoa_free = QAOA(
    problem=problems.MaxCut(G),
    mixer=mixers.XMultiAngle(),
    initialstate=initialstates.Plus(),
)

# Orbit QAOA: X mixer, one γ per edge orbit of the graph
qaoa_orbit = QAOA(
    problem=problems.MaxCutOrbit(G),
    mixer=mixers.X(),
    initialstate=initialstates.Plus(),
)

print(f"Vanilla  — γ/layer: {qaoa_vanilla.n_gamma}, β/layer: {qaoa_vanilla.n_beta}")
print(f"Free     — γ/layer: {qaoa_free.n_gamma},    β/layer: {qaoa_free.n_beta}")
print(f"Orbit    — γ/layer: {qaoa_orbit.n_gamma},   β/layer: {qaoa_orbit.n_beta}")

### Energy Landscape at Depth 1 (Vanilla)

Sample the energy landscape over (γ, β) for the vanilla instance.

In [ ]:
angles = {"gamma": [0, 2 * np.pi, 20], "beta": [0, np.pi, 20]}
qaoa_vanilla.sample_cost_landscape(angles=angles)

fig = plt.figure(figsize=(6, 5))
plot_E(qaoa_vanilla, fig=fig)
plt.title("Vanilla QAOA — energy landscape (depth 1)")
plt.show()

### Run Optimization

In [ ]:
maxdepth = 5

qaoa_vanilla.optimize(depth=maxdepth)
qaoa_free.optimize(depth=maxdepth, angles=angles)
qaoa_orbit.optimize(depth=maxdepth, angles=angles)

### Compare Approximation Ratios

In [ ]:
fig = plt.figure()

plot_ApproximationRatio(
    qaoa_vanilla, maxdepth,
    mincost=mincost, maxcost=maxcost,
    label="Vanilla (X mixer)",
    style="o--b",
    fig=fig,
)
plot_ApproximationRatio(
    qaoa_free, maxdepth,
    mincost=mincost, maxcost=maxcost,
    label="Free (XMultiAngle mixer)",
    style="s--r",
    fig=fig,
)
plot_ApproximationRatio(
    qaoa_orbit, maxdepth,
    mincost=mincost, maxcost=maxcost,
    label="Orbit (MaxCutOrbit + X mixer)",
    style="^--g",
    fig=fig,
)

plt.title("Approximation ratio vs. depth — House graph")
plt.tight_layout()
plt.show()

### Optimal Angles at Maximum Depth (Vanilla)

`plot_angles` expects the standard 1 γ + 1 β per-layer format.  
We display it for the vanilla variant; the free and orbit instances use a
different layout (multiple γ or β per layer).

In [ ]:
fig = plt.figure()
plot_angles(qaoa_vanilla, maxdepth, label="Vanilla", style="ob", fig=fig)
plt.title(f"Optimal angles — Vanilla QAOA, depth {maxdepth}")
plt.tight_layout()
plt.show()